# What is Multidimensional-Scaling?
Assume we have a 3 feature dataset with $N$ rows, we can represent all the points of this dataset in 3 dimensions. If we calculate the distance between each datapoint we end up with a distance matrix of shape $NxN$. With Multidimensional-scaling we are trying to represent this data in less dimensions (features) and getting a distance matrix as closest as the original. For this we use (as in PCA) linear algebra.

In this case we will try to transform a 3 feature dataset into 2 feature dataset. The process for multidimensional-scaling goes as follows:

1. Calculate the square-distance matrix $D^{2}$(remember this is a symmetric matix)

2. Apply double centering:
$$B = -\frac{1}{2}CD^{2}C$$

3. Get the eigenvalues and eigenvectors of $B$ and pick the $m$ largest eigenvalues ($m$ is the number of features we want to get).

4. The new dataset is described by:
$$X_{reduced} = E_{m}\Lambda_{m}^{1/2}$$ 

where $E_{m}$ is the matrix of $m$ eigenvectors and $\Lambda_{m}$ is the diagonal matrix of $m$ eigenvalues.

## Create dataset

In [38]:
# Lets create a dataset from sklearn.datasets.make_blob
## Create sample dataset with sklearn 
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

n_samples = 10
n_features = 3

data =  make_blobs(n_samples = n_samples, 
                   n_features = n_features, 
                   centers = 1,
                   random_state = 12)
feature_data = data[0]

## Calculate distance-saquared matrix

In [39]:
import numpy as np
D_squared = np.zeros((n_samples, n_samples))

for ii in range(n_samples):
    for jj in range(n_samples - ii - 1):
        D_squared[ii, ii + jj + 1] = np.linalg.norm(feature_data[ii] - feature_data[ii + jj + 1])**2
        D_squared[ii + jj + 1, ii] = D_squared[ii, ii + jj + 1] # Since the distance from A to B is the same from B to A 
                                                                # we don't need to calculate that again

## Apply double centering

In [40]:
Centering = np.identity(n_samples) - np.ones((n_samples, n_samples))/n_samples # Centering matrix

B = -Centering@D_squared@Centering/2

## Get eigenvectors and eigenvalues

In [41]:
eig_vals, eig_vec = np.linalg.eig(B)

## Get the top 2 eigenvalues and eigenvectors

In [42]:
max_pos = np.argsort(eig_vals)[-2:][::-1]

Em = np.transpose(eig_vec[max_pos])
Lambda = np.diag(eig_vals[max_pos])

X_reduced = Em@(Lambda**(1/2))

## Lets compare the distance matrices from the new X

In [43]:
DX_squared = np.zeros((n_samples, n_samples))

for ii in range(n_samples):
    for jj in range(n_samples - ii - 1):
        DX_squared[ii, ii + jj + 1] = np.linalg.norm(X_reduced[ii] - X_reduced[ii + jj + 1])**2
        DX_squared[ii + jj + 1, ii] = DX_squared[ii, ii + jj + 1] # Since the distance from A to B is the same from B to A 
                                                                # we don't need to calculate that again

## Calculate stress 

We can calculate the performance of the MDs by getting the stress, which basically is adding the difference of the distances obtained between all points from the original to the new feature-space divided by the original distance squared

In [45]:
stress = 0
for ii in range(n_samples):
    for jj in range(n_samples - ii - 1):
        stress += ((D_squared[ii, ii + jj + 1] - DX_squared[ii, ii + jj + 1])/D_squared[ii, ii + jj + 1])**2
stress

456.25860180434665

Since we are creating a random dataset, preserving distances might be difficult when reducing dimensions, nevertheless this method tries to keep the differences at minimum and a 0 is usually not possible. 